In [3]:
import json
import re
from collections import defaultdict

def tokenize(text):
    # تقسيم النص إلى كلمات بسيطة (يمكن تحسينها لاحقًا بإزالة علامات الترقيم والكلمات التوقف)
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

def build_inverted_index(docs):
    inverted_index = defaultdict(set)  # الكلمة -> مجموعة معرفات المستندات

    for doc in docs:
        doc_id = doc["doc_id"]
        text = doc["clean_text"]
        tokens = tokenize(text)
        unique_tokens = set(tokens)
        for token in unique_tokens:
            inverted_index[token].add(doc_id)

    # تحويل المجموعات لقوائم (لتسهيل التخزين أو المعالجة)
    inverted_index = {term: list(doc_ids) for term, doc_ids in inverted_index.items()}
    return inverted_index

def main():
    # قراءة الملف JSON
    path = r"C:\Users\Azzam\PycharmProjects\PythonProject\Data Pre-Processing\beir\quora\test\doc\docs.json"
    with open(path, 'r', encoding='utf-8') as f:
        docs = json.load(f)

    print(f"عدد المستندات: {len(docs)}")

    # بناء الفهرس
    inverted_index = build_inverted_index(docs)

    # مثال: عرض بعض المصطلحات وعدد مستنداتها
    for term in list(inverted_index.keys())[:10]:
        print(f"'{term}': {len(inverted_index[term])} مستندات")

    # حفظ الفهرس إلى ملف JSON
    with open('beir_inverted_index.json', 'w', encoding='utf-8') as f:
        json.dump(inverted_index, f, ensure_ascii=False, indent=2)
    print("✅ تم حفظ الفهرس في beir_inverted_index.json")

if __name__ == "__main__":
    main()


عدد المستندات: 522931
'step': 898 مستندات
'guide': 291 مستندات
'india': 17246 مستندات
'invest': 1105 مستندات
'market': 1906 مستندات
'share': 1171 مستندات
'i': 760 مستندات
'koh': 15 مستندات
'noor': 5 مستندات
'kohinoor': 12 مستندات
✅ تم حفظ الفهرس في beir_inverted_index.json


In [10]:
import os
import json
import joblib

chunks_path = r"C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks"
docid_to_chunk = {}

for chunk_file in os.listdir(chunks_path):
    if not chunk_file.startswith("hybrid_chunk_") or not chunk_file.endswith(".joblib"):
        continue
    chunk_path = os.path.join(chunks_path, chunk_file)
    chunk_data = joblib.load(chunk_path)
    for doc_id in chunk_data["doc_ids"]:
        docid_to_chunk[doc_id] = chunk_file

# حفظ الخريطة
with open(os.path.join(chunks_path, "docid_to_chunk.json"), "w", encoding="utf-8") as f:
    json.dump(docid_to_chunk, f, ensure_ascii=False, indent=2)

print(f"✅ تم بناء وحفظ خريطة doc_id إلى chunk_file في {chunks_path}\\docid_to_chunk.json")


✅ تم بناء وحفظ خريطة doc_id إلى chunk_file في C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks\docid_to_chunk.json


In [13]:
import json
import re
import os
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack


def custom_tokenizer(text):
    return text.split()

def tokenize(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# تحميل الفهرس العكسي
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

# تحميل موديل BERT
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_candidate_doc_ids(query_tokens):
    candidate_sets = []
    for token in query_tokens:
        if token in inverted_index:
            candidate_sets.append(set(inverted_index[token]))
    if candidate_sets:
        # اتحاد مجموعات المستندات التي تحتوي على أي من الكلمات
        candidates = set.union(*candidate_sets)
    else:
        candidates = set()
    return candidates

def load_chunks_for_docs_with_map(candidates, chunks_path, docid_to_chunk_path):
    # تحميل خريطة doc_id -> chunk_file
    with open(docid_to_chunk_path, 'r', encoding='utf-8') as f:
        docid_to_chunk = json.load(f)

    # خزن لكل chunk_file قائمة doc_ids المطلوبة
    chunk_to_docids = {}

    for doc_id in candidates:
        chunk_file = docid_to_chunk.get(doc_id)
        if chunk_file is not None:
            chunk_to_docids.setdefault(chunk_file, []).append(doc_id)

    tfidf_chunks = []
    bert_chunks = []
    doc_ids_all = []

    for chunk_file, doc_ids_needed in chunk_to_docids.items():
        chunk_path = os.path.join(chunks_path, chunk_file)
        chunk_data = joblib.load(chunk_path)
        chunk_doc_ids = chunk_data["doc_ids"]

        # جلب الإندكس للـ doc_ids المطلوبة ضمن هذا الـ chunk
        indices = [i for i, doc_id in enumerate(chunk_doc_ids) if doc_id in doc_ids_needed]

        if not indices:
            continue

        tfidf_chunk = chunk_data["tfidf_chunk"][indices]
        bert_chunk = chunk_data["bert_chunk"][indices]
        selected_doc_ids = [chunk_doc_ids[i] for i in indices]

        tfidf_chunks.append(tfidf_chunk)
        bert_chunks.append(bert_chunk)
        doc_ids_all.extend(selected_doc_ids)

    if tfidf_chunks:
        tfidf_matrix = vstack(tfidf_chunks)
        bert_matrix = np.vstack(bert_chunks)
    else:
        tfidf_matrix = None
        bert_matrix = None

    return tfidf_matrix, bert_matrix, doc_ids_all

def hybrid_search(query, vectorizer, chunks_path, docid_to_chunk_path, tfidf_weight=0.5, bert_weight=0.5, top_k=10):
    query_tokens = tokenize(query)

    candidates = get_candidate_doc_ids(query_tokens)
    if not candidates:
        print("⚠️ لا توجد مستندات مرشحة للفحص.")
        return []

    tfidf_matrix, bert_matrix, doc_ids = load_chunks_for_docs_with_map(candidates, chunks_path, docid_to_chunk_path)

    if tfidf_matrix is None or bert_matrix is None:
        print("⚠️ لم يتم تحميل تمثيلات لأي مستندات.")
        return []

    query_tfidf = vectorizer.transform([query])
    query_bert = bert_model.encode(query, convert_to_numpy=True).reshape(1, -1).astype(np.float32)

    tfidf_sim = cosine_similarity(tfidf_matrix, query_tfidf).flatten()
    bert_sim = cosine_similarity(bert_matrix, query_bert).flatten()

    final_scores = tfidf_weight * tfidf_sim + bert_weight * bert_sim

    top_indices = np.argsort(final_scores)[::-1][:top_k]
    results = [(doc_ids[i], float(final_scores[i])) for i in top_indices]

    return results

# --- استخدام الدالة ---




In [16]:
import joblib

# 1. استيراد الدالة اللي كتبناها (إذا في ملف منفصل)
# من فرضياتنا، الدالة hybrid_search موجودة هنا في نفس السكربت

# 2. تحميل vectorizer من ملف tfidf_data.joblib (في عندك الملف)
vectorizer_data = joblib.load(r"C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib")
vectorizer = vectorizer_data["vectorizer"]

# 3. تحديد مسار ملفات التمثيل الهجين (chunks)
chunks_path = r"C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks"
docid_to_chunk_path = os.path.join(chunks_path, "docid_to_chunk.json")

# 4. الاستعلام اللي بدنا نبحث فيه
query = "how to learn a amchine programming language?"

# 5. تنفيذ البحث الهجين
results = hybrid_search(
    query=query,
    vectorizer=vectorizer,
    chunks_path=chunks_path,
     docid_to_chunk_path=docid_to_chunk_path,
    tfidf_weight=0.4,
    bert_weight=0.6,
    top_k=10
)

# 6. طباعة النتائج
print("✅ نتائج البحث النهائية:")
for doc_id, score in results:
    print(f"Doc ID: {doc_id} | Score: {score:.4f}")


✅ نتائج البحث النهائية:
Doc ID: 91642 | Score: 0.6005
Doc ID: 527808 | Score: 0.5991
Doc ID: 498101 | Score: 0.5965
Doc ID: 44161 | Score: 0.5810
Doc ID: 1271 | Score: 0.5602
Doc ID: 112819 | Score: 0.5481
Doc ID: 364397 | Score: 0.5396
Doc ID: 9811 | Score: 0.5242
Doc ID: 22650 | Score: 0.5242
Doc ID: 419330 | Score: 0.5234
